# Bennets Hipathia Script

## 1. Instalation & Setup

In [ ]:
# --- Install 'dplyr' if missing ---
if (!requireNamespace("dplyr", quietly = TRUE)) {
    install.packages("dplyr")
}

# --- Install BiocManager if missing ---
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
}

# --- Install or update 'hipathia' from Bioconductor ---
BiocManager::install("hipathia", ask = FALSE, update = TRUE)

# --- Load core libraries ---
library(dplyr)
library(hipathia)
library(data.table)

# --- Check installed versions (optional but useful for documentation) ---
packageVersion("hipathia")
packageVersion("dplyr")


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.27), R 4.4.3 (2025-02-28)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'hipathia'”
Old packages: 'Biostrings', 'digest', 'gower', 'IRanges', 'mgcv', 'purrr',
  'rbibutils', 'reshape2', 'RSQLite', 'S7', 'SparseArray', 'xml2'

Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘digest’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘gower’ had non-zero exit status”


## 2. PARAMETERS

### 2.1 Set Paramter 

In [ ]:
# ---- Input files ----
rna_path  <- "rna_data_filtered wtumor.csv"        # raw RNA matrix: Genes × Cells
meta_path <- "complet_cell_metadata_grouped.csv"   # metadata with cell_id + celltype info

# ---- Output files ----
rna_standardized_out <- "rna_standardized.csv"
expr_pseudobulk_out  <- "expr_pseudobulk_symbols.csv"
design_out           <- "design.csv"

# Column in metadata that contains celltype annotation
group_column <- "celltype"
# ---- DEFINE GROUPS YOU WANT TO COMPARE ----
group1_name  <- "Normal"        # name in output (left in Hipathia)
group2_name  <- "Tumor"        # name in output (right in Hipathia)
# Which celltypes belong to each group?
group1_types <- c(" Mono_CD14")     # OR: c("CD4 T_cells")
group2_types <- c("Malignant")       # OR: c("CD8 T_cells")

# 2.3 Sample ID to inspect for celltype composition
inspect_sample <- "p4"

# Combined list of valid groups
allowed_groups <- c(group1_types, group2_types)

# ---- Options ----
pseudobulk_method <- "sum"
min_overlap_prop  <- 0.9
min_cells     <- 30

# ---- Hipathia ----
species       <- "hsa"
pathways_list <- c()   # empty = load all pathways





## PATIENT FILTERING

In [ ]:
# Load metadata
meta <- read.csv(meta_path, stringsAsFactors = FALSE, check.names = FALSE)

# Ensure required columns exist
stopifnot(all(c("cell_id", "orig.ident", group_column) %in% colnames(meta)))

# Merge pXXXpos / pXXXneg → pXXX (patient-level)
meta$patient_id <- gsub("pos|neg", "", meta$orig.ident)

# Filter metadata to keep only the groups defined in the parameter block
meta_filt <- meta[meta[[group_column]] %in% c(group1_types, group2_types), ]

# Count cells per patient × group
cell_counts <- table(meta_filt$patient_id, meta_filt[[group_column]])

# Universal: sum counts for each group1/group2 type (in case multiple celltypes belong to a group)
group1_counts <- rowSums(cell_counts[, group1_types, drop = FALSE])
group2_counts <- rowSums(cell_counts[, group2_types, drop = FALSE])

# Which patients have enough cells from BOTH groups?
valid_patients <- names(which(group1_counts >= min_cells & group2_counts >= min_cells))


# ============================================
# BUILD GROUPED PATIENT × CELLTYPE TABLE
# ============================================

patients <- rownames(cell_counts)

# Reihen in der richtigen Reihenfolge
rows <- c(
  paste0('group1_name = "', group1_name, '"'),
  group1_types,
  paste0('group2_name = "', group2_name, '"'),
  group2_types,
  "keep"
)

# Matrix erstellen
diagnosis_matrix <- matrix(
  "",
  nrow = length(rows),
  ncol = length(patients),
  dimnames = list(rows, patients)
)

# Summen für group1
for (p in patients) {
  diagnosis_matrix[paste0('group1_name = "', group1_name, '"'), p] <-
    sum(cell_counts[p, group1_types], na.rm = TRUE)
}

# Summen für group2
for (p in patients) {
  diagnosis_matrix[paste0('group2_name = "', group2_name, '"'), p] <-
    sum(cell_counts[p, group2_types], na.rm = TRUE)
}

# Einzelne Celltypes einfügen
for (ct in group1_types) {
  diagnosis_matrix[ct, ] <- as.character(cell_counts[, ct])
}
for (ct in group2_types) {
  diagnosis_matrix[ct, ] <- as.character(cell_counts[, ct])
}

# keep-Zeile
diagnosis_matrix["keep", ] <- ifelse(
  group1_counts >= min_cells & group2_counts >= min_cells,
  "TRUE",
  "FALSE"
)


# In data.frame konvertieren
diagnosis_table <- as.data.frame(diagnosis_matrix)


# Print diagnostic table
print(diagnosis_table)

cat("\n===== PATIENT FILTERING DONE =====\n")
cat("Comparison: ", group1_name, " vs ", group2_name, "\n", sep = "")
cat("Min cells per group required: ", min_cells, "\n", sep = "")
cat("Patients passing filter: ", paste(valid_patients, collapse = ", "), "\n\n", sep = "")

# Final metadata restricted to valid patients only
meta_final <- meta_filt[meta_filt$patient_id %in% valid_patients, ]

# Assign Normal/Tumor group to each cell based on its celltype
meta_final$group <- ifelse(
    meta_final[[group_column]] %in% group1_types,
    group1_name,
    group2_name
)

# After filtering patients: extract valid cell barcodes
filtered_barcodes <- unique(meta_final$cell_id)

if (length(filtered_barcodes) == 0) {
    stop("Patient filter produced zero remaining cell barcodes.")
}

message("Number of remaining barcodes after filtering: ", length(filtered_barcodes))



## 3. RNA Reader & Standardisation

### 3.1 Helper functions

In [ ]:

trim_all <- function(df) {
  names(df) <- trimws(names(df))
  for (i in seq_along(df)) {
    if (is.character(df[[i]]))
      df[[i]] <- trimws(df[[i]])
  }
  df
}

looks_gene_symbol <- function(v) {
  v <- trimws(as.character(v))
  non_na <- v[!is.na(v)]

  if (length(non_na) == 0) return(FALSE)

  return(
    mean(grepl("[A-Za-z]", non_na)) > 0.5 &&
    mean(nchar(non_na) < 20) > 0.95 &&
    mean(grepl("-", non_na)) < 0.5 &&
    mean(grepl("^\\d+$", non_na)) < 0.1
  )
}

looks_barcode <- function(x) {
  x <- trimws(as.character(x))
  grepl("^[ACGTN]+[-_][0-9]+$", x)
}

# ---- WICHTIG: Die neue fread-basierte read_tab Funktion ----
read_tab <- function(path) {
  fread(path, data.table = FALSE)
}


### 3.2. Read RNA File

In [ ]:
rna_raw <- read_tab(rna_path)
rna_raw <- trim_all(rna_raw)

rna_ids <- colnames(rna_raw)
meta_ids <- meta$cell_id

cat("Meta cells:", length(meta_ids), "\n")
cat("RNA cells: ", length(rna_ids), "\n")

cat("Cells in metadata but NOT in RNA:", 
    sum(!(meta_ids %in% rna_ids)), "\n")

# Drop empty columns
empty_cols <- which(sapply(rna_raw, function(x) all(x == "" | is.na(x))))
if (length(empty_cols)) rna_raw <- rna_raw[, -empty_cols, drop = FALSE]

stopifnot(ncol(rna_raw) >= 2)

# Detect gene column (robust)
# 1. Check common gene column names first
gene_name_hits <- which(tolower(colnames(rna_raw)) %in% c("gene", "genes", "symbol", "geneid"))

if (length(gene_name_hits) == 1) {
    gene_col <- gene_name_hits
} else {
    # 2. Fallback: heuristic
    first_col <- rna_raw[[1]]
    if (looks_gene_symbol(head(first_col, 200))) {
        gene_col <- 1
    } else {
        gene_col <- which(sapply(rna_raw, function(col)
            looks_gene_symbol(head(col, 200))))[1]
    }
}

if (is.na(gene_col)) stop("Could not detect gene column.")

colnames(rna_raw)[gene_col] <- "Gene"


### Check: cells per patient X


In [ ]:
# ---- Inspect patient cell composition ----

cat("\n===== Inspecting sample:", inspect_sample, "=====\n")

# Filter for selected patient
meta_inspect <- meta[meta$orig.ident == inspect_sample, ]

if (nrow(meta_inspect) == 0) {
    stop("No cells found for patient: ", inspect_sample)
}

# Unique cell types
unique_celltypes_inspect <- unique(meta_inspect[[group_column]])

cat("\nUnique cell types for patient", inspect_sample, ":\n")
print(unique_celltypes_inspect)

# Count per celltype
table_inspect <- table(meta_inspect[[group_column]])

cat("\nCell counts for patient", inspect_sample, ":\n")
print(table_inspect)

cat("\n===== End inspection for:", inspect_sample, "=====\n\n")


### 3.3 Standardization RNA Matrix

In [ ]:
rna <- rna_raw

# Gene column
rna$Gene <- as.character(rna$Gene)

# Welche Barcodes dürfen bleiben?
rna_barcodes   <- setdiff(colnames(rna), "Gene")
keep_barcodes  <- intersect(rna_barcodes, filtered_barcodes)

message("Keeping ", length(keep_barcodes), " barcodes after patient filtering.")

# Relevante Spalten auswählen
rna <- rna[, c("Gene", keep_barcodes), drop = FALSE]

# Bessere Numeric-Erkennung:
# akzeptiert: 1, 1.5, 1e-5, 0E0, NA, "", " "
valid_numeric <- function(x) {
    x <- trimws(x)
    ok <- grepl("^[-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?$", x) | x == "" | is.na(x)
    all(ok)
}

# Konvertiere Barcodes
for (bc in keep_barcodes) {
    if (!valid_numeric(rna[[bc]])) {
        stop("Column ", bc, " contains non-numeric entries. This should not happen in an RNA matrix.")
    }

    rna[[bc]] <- suppressWarnings(as.numeric(trimws(rna[[bc]])))
}

# Drop empty genes
rna$Gene <- trimws(rna$Gene)
rna <- rna[rna$Gene != "", , drop = FALSE]

# Collapse duplicate genes (WICHTIG: keine Sortierung + Barcodes bleiben korrekt)
if (anyDuplicated(rna$Gene)) {
    message("Duplicate genes detected → collapsing by sum.")

    # Aggregiere korrekt und erhalte Barcode-Namen
    rna <- aggregate(
        rna[, keep_barcodes, drop = FALSE],
        by = list(Gene = rna$Gene),
        FUN = sum
    )

    # Stellt sicher, dass Reihenfolge Gene, Barcode1, Barcode2,... bleibt
    rna <- rna[, c("Gene", keep_barcodes), drop = FALSE]
}

# Export
write.csv(rna, rna_standardized_out, row.names = FALSE)
message("Wrote standardized RNA: ", ncol(rna) - 1, " barcodes kept.")


### 3.4. Check: Barcode Overlap AFTER standardization

In [ ]:


meta_ids <- meta_final$cell_id             # nur Patienten, die du behalten hast
rna_ids  <- setdiff(colnames(rna), "Gene") # alle Barcodes, die noch in RNA sind

overlap <- intersect(meta_ids, rna_ids)
prop <- length(overlap) / length(rna_ids)

cat("\n===== Barcode Overlap AFTER standardization =====\n")
cat("Remaining RNA barcodes:   ", length(rna_ids), "\n")
cat("Remaining meta barcodes:  ", length(meta_ids), "\n")
cat("Overlap:                  ", length(overlap),
    sprintf(" (%.1f%%)\n", 100 * prop))

if (prop < min_overlap_prop) {
    warning("Barcode overlap after filtering < ",
            100 * min_overlap_prop, "% — check filtering logic.")
}


### CHECK: Patient filter sanity

In [ ]:
# ==============================
# CHECK: Patient filter sanity
# ==============================

message("\n===== PATIENT FILTER CHECK =====")

# 1) Anzahl der Barcodes
message("Barcodes after filtering: ", length(filtered_barcodes))

if (length(filtered_barcodes) == 0) {
    stop("ERROR: No barcodes kept after patient filtering.")
}

# 2) Prüfen ob Barcodes in der RNA-Datei existieren
barcode_cols_in_rna <- setdiff(colnames(rna_raw), "Gene")

missing_in_rna <- setdiff(filtered_barcodes, barcode_cols_in_rna)

if (length(missing_in_rna) > 0) {
    message("WARNING: ", length(missing_in_rna),
            " barcodes are in metadata but missing in the RNA matrix:")
    print(head(missing_in_rna))
} else {
    message("All filtered barcodes exist in the RNA matrix. ✔")
}

# 3) Prüfen ob RNA-Barcodes existieren, die NICHT gefiltert wurden
extra_in_rna <- setdiff(barcode_cols_in_rna, filtered_barcodes)
message("RNA barcodes not part of filtered patients: ", length(extra_in_rna))


In [ ]:
# 1. Number of barcodes in the original RNA matrix
original_rna_barcodes <- setdiff(colnames(rna_raw), "Gene")
message("Original RNA barcodes: ", length(original_rna_barcodes))

# 2. Number of barcodes after metadata filtering
message("Filtered (kept) barcodes in metadata: ", length(filtered_barcodes))

# 3. How many barcodes match between RNA and metadata?
meta_barcodes <- meta_final$cell_id
message("Overlap (RNA ∩ metadata): ",
        length(intersect(original_rna_barcodes, meta_barcodes)))

# 4. Which RNA barcodes were NOT kept by the patient filter?
lost <- setdiff(original_rna_barcodes, filtered_barcodes)
message("RNA barcodes discarded by patient filter: ", length(lost))
if (length(lost) > 0) print(head(lost, 20))




## 4. Pseudobulk 

In [ ]:
message("\n===== BUILDING PSEUDOBULK (Patient × Group) =====")

# 1) Keep filtered barcodes that exist in RNA
barcode_cols <- intersect(colnames(rna), filtered_barcodes)

rna_sub <- rna[, c("Gene", barcode_cols), drop = FALSE]

# 2) Filter metadata
meta_sub <- meta_final[meta_final$cell_id %in% barcode_cols, ]

# 3) Assign groups
meta_sub$group <- ifelse(
    meta_sub[[group_column]] %in% group1_types,
    group1_name,
    group2_name
)

# 4) Build unique sample names: Patient + "_" + Group
meta_sub$sample_group <- paste0(meta_sub$patient_id, "_", meta_sub$group)

sample_groups <- unique(meta_sub$sample_group)

# 5) Build pseudobulk for each Patient×Group
pb_mat <- sapply(sample_groups, function(sg) {
    cells <- meta_sub$cell_id[meta_sub$sample_group == sg]
    r_sub <- rna_sub[, cells, drop = FALSE]
    rowSums(r_sub, na.rm = TRUE)
})

# 6) Construct final pseudobulk dataframe
expr_pseudobulk <- data.frame(
    Gene = rna_sub$Gene,
    pb_mat,
    check.names = FALSE
)

# 7) Build design table
design <- data.frame(
    sample_id = sample_groups,
    patient   = sapply(sample_groups, function(x) strsplit(x, "_")[[1]][1]),
    group     = sapply(sample_groups, function(x) strsplit(x, "_")[[1]][2])
)

# 8) Save
write.csv(expr_pseudobulk, expr_pseudobulk_out, row.names = FALSE)
write.csv(design, design_out, row.names = FALSE)

message("Pseudobulk written for ", length(sample_groups), " sample groups.")
                       table(meta_sub$patient_id, meta_sub$group)



## Rest


In [ ]:
## ==============================
## Step 1: Map Gene Symbols to Entrez IDs  (fixed root version)
## ==============================

# Install Bioconductor annotation package if missing
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

if (!requireNamespace("org.Hs.eg.db", quietly = TRUE))
    BiocManager::install("org.Hs.eg.db")

# Load library
library(org.Hs.eg.db)

# Read pseudobulk matrix (symbols)
expr_pb <- read.csv(expr_pseudobulk_out, stringsAsFactors = FALSE, check.names = FALSE)

# Check
if (!"Gene" %in% colnames(expr_pb)) stop("Expression file must contain column 'Gene'.")

# Map Gene Symbols -> Entrez IDs
gene_symbols <- expr_pb$Gene
entrez_ids <- mapIds(org.Hs.eg.db,
                     keys = gene_symbols,
                     column = "ENTREZID",
                     keytype = "SYMBOL",
                     multiVals = "first")

# Add Entrez IDs
expr_pb$EntrezID <- entrez_ids

# Remove genes without mapping
expr_pb <- expr_pb[!is.na(expr_pb$EntrezID), ]

# Collapse duplicates (mean per EntrezID)
expr_pb <- aggregate(. ~ EntrezID, data = expr_pb[, c("EntrezID", setdiff(colnames(expr_pb), c("Gene", "EntrezID")))],
                     FUN = function(x) if (is.numeric(x)) mean(as.numeric(x), na.rm = TRUE) else x[1])

# Use EntrezID as rownames
rownames(expr_pb) <- expr_pb$EntrezID

# Remove redundant column
expr_pb$EntrezID <- NULL

# Save clean expression matrix
write.csv(expr_pb, "expr_pseudobulk_entrez.csv", row.names = TRUE)

# Report
cat("\n===== MAPPING DONE (fixed root version) =====\n")
cat("Input genes (symbols): ", length(gene_symbols), "\n", sep = "")
cat("Mapped to Entrez IDs : ", nrow(expr_pb), "\n", sep = "")
cat("File written: expr_pseudobulk_entrez.csv\n")


In [ ]:
#3.1
library(hipathia)

trans_data <- as.matrix(read.csv("expr_pseudobulk_entrez.csv",
                                 row.names = 1, check.names = FALSE))



In [ ]:
#3.2
# exakt der Vignetten-Schritt:
exp_data <- normalize_data(trans_data, by_quantiles = FALSE)

write.csv(data.frame(Gene=rownames(exp_data), exp_data, check.names=FALSE),
          "Vinette_Like_RNA-Data.csv", row.names=FALSE)


In [ ]:
# vor Scaling
boxplot(trans_data)

# Limit threads (verhindert pthread-Fehler auf Clustern)
Sys.setenv(OMP_NUM_THREADS="1", MKL_NUM_THREADS="1",
           OPENBLAS_NUM_THREADS="1", VECLIB_MAXIMUM_THREADS="1")

# Versuche Quantile-Norm, sonst nimm Truncation wie in der Vignette
exp_data <- tryCatch(
  normalize_data(trans_data, by_quantiles = TRUE),
  error = function(e) {
    message("Quantile normalization failed (preprocessCore). Using truncation_percentil = 0.95.")
    normalize_data(trans_data, truncation_percentil = 0.95)
  }
)

boxplot(exp_data)


In [ ]:
#4.1
if (length(pathways_list) == 0) {
    message("No pathway list provided -> loading ALL pathways for ", species)
    pathways <- load_pathways(species = species)
} else {
    message("Loading subset of pathways: ", paste(pathways_list, collapse = ", "))
    pathways <- load_pathways(species = species, pathways_list = pathways_list)
}

# ---- Check
length(pathways)   # Anzahl geladener Pathways
names(pathways)[1:5]  # Erste 5 Pathways anzeigen
get_pathways_list(pathways)[1:10]

In [ ]:
#4.3 Using Hipathia to comute the signal
results <- hipathia(exp_data, pathways, decompose = FALSE, verbose=FALSE)
## Added missing genes: 165 (4.93%)

In [ ]:
#4.3 Using Hipathia to comute the signal
results

In [ ]:
#4.3 Using Hipathia to comute the signal
path_vals <- get_paths_data(results)
hhead(path_vals, 4)

#5 Function activation computation
uniprot_vals <- quantify_terms(results, pathways, dbannot = "uniprot")

go_vals <- quantify_terms(results, pathways, dbannot = "GO")
#142/1654 soll so sein

In [ ]:
#6.1  Two classes comparison
design <- read.csv(meta_path, stringsAsFactors = FALSE, check.names = FALSE)
design <- design[!duplicated(design$orig.ident), c("orig.ident","group")]
rownames(design) <- design$orig.ident

sample_group <- design[colnames(path_vals), "group"]
comp <- do_wilcoxon(path_vals[, names(sample_group), drop = FALSE], sample_group, g1 = "Tumor", g2 = "Normal")
head(comp)


In [ ]:
#6.1  Two classes comparison
pathways_summary <- get_pathways_summary(comp, pathways)
head(pathways_summary, 4)

In [ ]:
#6.2 Pricipal Components ANalysis
ranked_path_vals <- path_vals[order(comp$p.value, decreasing = FALSE),]
pca_model <- do_pca(ranked_path_vals[1:ncol(ranked_path_vals),])

In [ ]:
#7Heatmap
heatmap_plot(path_vals, group = sample_group)


In [ ]:
# This should be TRUE; if FALSE, you sorted columns/rows inkonsistent
all(rownames(ranked_path_vals) %in% rownames(path_vals))
all(rownames(ranked_path_vals)[1:5] == rownames(comp)[order(comp$p.value, decreasing=FALSE, na.last=NA)][1:5])


In [ ]:
heatmap_plot(uniprot_vals, group = sample_group, colors="hipathia",
variable_clust = TRUE)
heatmap_plot(go_vals, group = sample_group, colors="redgreen",
variable_clust = TRUE)

In [ ]:
pca_plot(pca_model, sample_group, legend = TRUE)

In [ ]:
pca_plot(pca_model, sample_group, legend = TRUE)

In [ ]:

pca_plot(pca_model, group = rep(1:5, 8), main = "Random types",
legend = TRUE)

In [ ]:
multiple_pca_plot(pca_model, sample_group, cex=3, plot_variance = TRUE)

In [ ]:
pathway_comparison_plot(comp, metaginfo = pathways, pathway = "hsa03320")

In [ ]:
colors_de <- node_color_per_de(results, pathways, sample_group, "Tumor",
"Normal")
pathway_comparison_plot(comp, metaginfo = pathways, pathway = "hsa03320",
node_colors = colors_de)

In [ ]:
colors_de_hipathia <- node_color_per_de(results, pathways, sample_group,
"Tumor", "Normal", colors = "hipathia")
pathway_comparison_plot(comp, metaginfo = pathways, pathway = "hsa03320",
node_colors = colors_de_hipathia, colors = "hipathia")
pathway_comparison_plot(
  comp,
  metaginfo = pathways,
  pathway = "hsa03320",
  node_colors = colors_de_hipathia,
  colors = "hipathia",
)

In [ ]:
#7.4. Vizualization through a local server
report <- create_report(comp, pathways, "save_noColors")
## Creating report folders...
## Creating pathways folder...
## Creating HTML index...
#report_colors <- create_report(comp, pathways, "save_colors",
#node_colors = colors_de)
## Creating report folders...
## Creating pathways folder...
## Creating HTML index...


In [ ]:

visualize_report(report, port = 4002)

In [ ]:
# 1) alte servr-Prozesse beenden
if (requireNamespace("servr", quietly = TRUE)) {
  dl <- try(servr::daemon_list(), silent = TRUE)
  if (!inherits(dl, "try-error") && NROW(dl)) {
    for (i in dl$id) try(servr::daemon_stop(i), silent = TRUE)
  }
}

# 2) freien Port wählen
port <- servr::random_port()  # nimmt garantiert freien Port, z.B. 4010

# 3) Report visualisieren
visualize_report(report, port = port)


In [ ]:
apply(trans_data, 2, sd)


In [ ]:
comp$FDRp.value <- runif(nrow(comp), 0, 0.01)  # zufällige kleine Werte
comp$UP.DOWN <- sample(c("UP","DOWN"), nrow(comp), replace = TRUE)

pathway_comparison_plot(
  comp,
  metaginfo = pathways,
  pathway = "hsa03320",
  node_colors = node_color_per_de(results, pathways, sample_group,
                                  "Tumor", "Normal", colors = "hipathia"),
  colors = "hipathia"
)


In [ ]:
apply(trans_data, 2, sd)


In [ ]:
table(design$group)


In [ ]:
table(meta$group, meta$orig.ident)
